In [1]:
import pandas as pd
import gzip
import json
import nlp_lib as nl
from sklearn.feature_extraction.text import CountVectorizer

## Data Loading and Cleaning

In [2]:
# Create generator function from raw data file to be later loaded into a data frame
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

In [3]:
# Iterate through generator and load data, line by line, into new dictionary
def get_reviews(path):
  reviews = []
  for d in parse(path):
    reviews.append(nl.Reviews(d['reviewText'], d['overall']))
  return reviews

In [4]:
# Create new dictionary with all the data
reviews = get_reviews("data/Appliances_5.json.gz")

## Create "Bag of Words" Feature Matrix for Training/Test Data

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
# Use sklearn to split up data into training and test splits, with a seed value to recreate from this state
review_train_set, review_test_set = train_test_split(reviews, train_size=0.6, random_state=9)

In [9]:
# split the features based on X and y (input data and class) for both the training and test sets
review_train_feat_x = [x.review_text for x in review_train_set]
review_train_feat_y = [y.sentiment for y in review_train_set]

review_test_feat_x = [x.review_text for x in review_test_set]
review_test_feat_y = [y.sentiment for y in review_test_set]

In [10]:
# Create new Count Vectorizer object and "fit" the vectorizer vocabulary to the training data.
# Constructs the feature columns (words in all training examples)
review_vectorizer = CountVectorizer()
review_vectorizer.fit(review_train_feat_x)

# Now that the vocabulary is known, transform training/test feature lists into BoW matrices
# row dim = # of training examples; col dim = size of vocabulary (unique word per column)
review_train_mat_x = review_vectorizer.transform(review_train_feat_x)
review_test_mat_x = review_vectorizer.transform(review_test_feat_x)

## Train model(s)

### Logistic Regression

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# Create the logistic regression class object and fit it to the training data
log_reg = LogisticRegression(random_state=9)
log_reg_model = log_reg.fit(review_train_mat_x, review_train_feat_y)

In [12]:
# Show the accuracy of the trained model on training and test data
print('train = {}'.format(log_reg_model.score(review_train_mat_x, review_train_feat_y)))
print('test = {}'.format(log_reg_model.score(review_test_mat_x, review_test_feat_y)))

train = 0.9970717423133236
test = 0.9901207464324918


In [13]:
# Show the F1 score for each classifier in the model (not good for negative, not enough examples)
log_f1 = f1_score(review_test_feat_y, log_reg_model.predict(review_test_mat_x), average=None, labels=['POSITIVE', 'NEGATIVE', 'NEUTRAL'])

In [14]:
log_f1

array([0.99392302, 0.54545455, 0.98787879])

In [16]:
review_train_feat_y.count('NEUTRAL')

254